In [35]:
import os 
import shutil
import glob
import random
import torch
from torchvision import transforms
from torch.utils.data import Dataset
from torchvision import datasets
import re
from PIL import Image
import numpy as np
from torch.utils.data import DataLoader
from torch import nn
from torchvision import datasets
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F

In [22]:
random.seed(42)
path = os.getcwd()
classes =  ["Circle", "Square", "Octagon","Heptagon", "Nonagon", "Star", "Hexagon", "Pentagon", "Triangle"]

training_files = glob.glob(os.path.join(path,"geometry_dataset\training_images\*.png"))
test_files = glob.glob(os.path.join(path,"geometry_dataset\test_images\*.png"))

for training_file,test_file in zip(training_files,test_files):
    os.remove(training_file)
    os.remove(test_file)

for class_ in classes:
    image_location = os.path.join(path,"geometry_dataset\output")
    image_location = os.path.join(image_location,class_ + "*.png")
    class_images = glob.glob(image_location)
    print(class_,len(class_images))
    random.shuffle(class_images)
    for train_images in class_images[0:8000]:
        shutil.copy(train_images,"geometry_dataset/training_images/"+str(class_))
    for test_images in class_images[8000:]:
        shutil.copy(test_images,"geometry_dataset/test_images/"+str(class_))

In [23]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1    = nn.Conv2d(3, 32, 3, 1)
        self.conv2    = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1      = nn.Linear(614656, 128)
        self.fc2      = nn.Linear(128, 9)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

In [38]:
def train(batch_size, model, device, train_loader, optimizer, epoch):
    model.train()
    tot_loss = 0
    correct  = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss   = torch.nn.CrossEntropyLoss()(output, target)
        loss.backward()
        optimizer.step()

        pred    = output.argmax(dim=1, keepdim=True) 
        correct += pred.eq(target.view_as(pred)).sum().item()

        tot_loss = tot_loss + loss.item()
        if batch_idx % 50 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}, Accuracy: {:.2f}%'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), tot_loss/(batch_idx+1), 100.0*correct/((batch_idx+1)*batch_size)))

    loss = tot_loss / len(train_loader)
    acc = 100.0 * correct / (len(train_loader) * batch_size)
    print('End of Epoch: {}'.format(epoch))
    print('Training Loss: {:.6f}, Training Accuracy: {:.2f}%'.format(loss, acc))

In [25]:
def test(batch_size, model, device, test_loader):
    model.eval()
    tot_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output       = model(data)
            tot_loss     += torch.nn.CrossEntropyLoss()(output, target).item()  # sum up batch loss
            pred         = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct      += pred.eq(target.view_as(pred)).sum().item()

    loss = tot_loss / len(test_loader)
    acc = 100.0 * correct / (len(test_loader) * batch_size)
    print('Test Loss: {:.6f}, Test Accuracy: {:.2f}%'.format(loss, acc))
    return loss, acc

In [39]:
batch_size = 100
epochs = 1

transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
train_dataset = datasets.ImageFolder(os.path.join(os.getcwd(),"geometry_dataset/training_images"),transform=transform)
test_dataset = datasets.ImageFolder(os.path.join(os.getcwd(),"geometry_dataset/test_images"),transform=transform)
train_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
test_loader = DataLoader(train_dataset,batch_size=batch_size,shuffle=True)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = Net().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)

scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, epochs + 1):
    train(batch_size, model, device, train_loader, optimizer, epoch)
    test_loss, test_acc = test(batch_size,model, device, test_loader)
    scheduler.step()

    if args.save_model: # save new model after every epoch
        torch.save(model.state_dict(), "0602-656377418-Garg.pt")

    if test_loss < 1e-4 or math.isclose(test_acc, 100.0):
        break

Train Epoch: 1 [0/72000 (0%)]	Loss: 2.202981, Accuracy: 13.00%
Train Epoch: 1 [5000/72000 (7%)]	Loss: 5.715325, Accuracy: 10.63%
Train Epoch: 1 [10000/72000 (14%)]	Loss: 3.974327, Accuracy: 10.69%
Train Epoch: 1 [15000/72000 (21%)]	Loss: 3.386461, Accuracy: 11.03%
Train Epoch: 1 [20000/72000 (28%)]	Loss: 3.090663, Accuracy: 11.21%
Train Epoch: 1 [25000/72000 (35%)]	Loss: 2.912855, Accuracy: 11.15%
Train Epoch: 1 [30000/72000 (42%)]	Loss: 2.794087, Accuracy: 11.07%


KeyboardInterrupt: 